# fucntion calling
## basic knowledge

![function-calling flow chart](./imgs/flow-chart.svg)

| Step | Executor   | Key Action                     | Data Format Example                                           |
| :--- | :--------- | :----------------------------- | :------------------------------------------------------------ |
| 1    | Application | Send user input + function definitions | `{"messages":[...], "functions":[...]}`                      |
| 2    | Large Model | Decide response method         | Generate `tool_calls` or respond directly                     |
| 3    | Large Model | Return function call           | `{"tool_calls": [{"name": "func", "arguments": {...}}]}`     |
| 4    | Application | Execute local function         | Parse arguments and invoke `func(**args)`                    |
| 5    | Application | Submit execution result        | `{"tool_call_id": "...", "role": "tool", "content": "result"}` |

Chinese vesion:
| 步骤 | 执行方 | 关键动作              | 数据格式示例                                                 |
| :--- | :----- | :-------------------- | :----------------------------------------------------------- |
| 1    | 应用   | 发送用户输入+函数定义 | `{"messages":[...], "functions":[...]}`                      |
| 2    | 大模型 | 决策响应方式          | 生成`tool_calls`或 直接回答                                   |
| 3    | 大模型 | 返回函数调用          | `{"tool_calls": [{"name": "func", "arguments": {...}}]}`     |
| 4    | 应用   | 执行本地函数          | 解析参数并调用`func(**args)`                                 |
| 5    | 应用   | 提交执行结果          | `{"tool_call_id": "...", "role": "tool", "content": "result"}` |

step 5 is optional

addional flowchart to understand it better

 <img src="./imgs/flow-chart2.svg" alt="Flow Chart" width="400" />

## basic usage

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
def get_weather(location):
    return "24℃"


available_functions = {"get_weather": get_weather}

In [ ]:
from openai import OpenAI


def send_messages(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message


client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

In [ ]:
messages = [{"role": "user", "content": "How's the weather in Hangzhou?"}]
message = send_messages(messages)
messages.append(message)
print(f"User>\t {messages[0]['content']}")
print(f"model response with tool_to_call\n: {message}")

tool = message.tool_calls[0]

User>	 How's the weather in Hangzhou?
model response with tool_to_call: ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)])


观察上面打印的LLM结果，其中`content`为空，`tool_calls`不空，表示LLM告诉我们下一步需要调用对应的tools。相反，如果LLM返回的是直接结果，则content不空，tool_calls为空。

In [ ]:
# for beginners, to understand the data structure
print(tool)
print(tool.function.name)
print(tool.function.arguments)
print(type(tool.function.arguments))

ChatCompletionMessageToolCall(id='call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)
get_weather
{"location":"Hangzhou"}
<class 'str'>


In [ ]:
import json
function_name = available_functions[tool.function.name]
function_args = json.loads(tool.function.arguments)
function_response = function_name(**function_args)  # call the function
function_response

'24℃'

* 为什么要传 `tool_call_id`： 可能会涉及到多个tool的调用，为了让LLM匹配哪个tool的结果是什么。
* 关于role，我们要注意，本地function执行结果使用`role: tool`返回给LLM

In [ ]:
# append the function response into the messages, which will be sent to the model to let model know the result of the tool
messages.append({"role": "tool", "tool_call_id": tool.id,
                "content": function_response})
message = send_messages(messages)
print(f"Model>\t {message.content}")
messages.append(message)

Model>	 The current weather in Hangzhou is 24°C. It's a pleasant temperature!


In [ ]:
messages

[{'role': 'user', 'content': "How's the weather in Hangzhou?"},
 ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)]),
 {'role': 'tool',
  'tool_call_id': 'call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae',
  'content': '24℃'},
 ChatCompletionMessage(content="The current weather in Hangzhou is 24°C. It's a pleasant temperature!", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)]

## parallel usage


| 模式                      | 行为                                                                 |
|---------------------------|----------------------------------------------------------------------|
| parallel_tool_calls: true  | 模型可以一次性发出多个工具请求（比如：同时查询天气、翻译一段话、查汇率）。 |
| parallel_tool_calls: false | 模型一次只能发出一个请求，用完一个工具后再继续发下一个。               |



function calling默认可以并行执行(`parallel_tool_calls: True`)，即一个流程中可调用多个函数，或调用同一个函数多次。
`parallel_tool_calls: False`时，这次会话只能调用一次函数

In [1]:
from dotenv import load_dotenv
load_dotenv()


def get_weather(location):
    if location.lower() == "hangzhou":
        return "24℃"
    elif location.lower() == "shanghai":
        return "25℃"
    else:
        return "30℃"


available_functions = {"get_weather": get_weather}

In [ ]:
from openai import OpenAI


def send_messages(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        # parallel_tool_calls = True
    )
    return response.choices[0].message


client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

In [3]:
messages = [
    {"role": "user", "content": "How's the weather in Hangzhou and Shanghai?"}]
message = send_messages(messages)

In [4]:
message.tool_calls

[ChatCompletionMessageToolCall(id='call_G8pfJhiyq9KoNwBQLU0Meezl', function=Function(arguments='{"location": "Hangzhou, China"}', name='get_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_F5pQPh33Oc3ikpyQSki2Rqmj', function=Function(arguments='{"location": "Shanghai, China"}', name='get_weather'), type='function')]

从上面看出，每个function calling实例都对应有一个tool id，及时调用相同的函数，对应的tool id是不同的。

下面演示parallel_tool_calls=False。

这个实验失败了，运行效果与true相同。
期望的运行效果为，只识别第一个参数，而忽略之后的参数。例如，询问杭州与上海的天气，只识别杭州。TODO

[deepsearch](https://chatgpt.com/share/680f4ba0-6654-800c-bd50-5b4ac1009d40)

In [3]:
def send_messages(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        parallel_tool_calls=False,
    )
    return response.choices[0].message


messages = [
    {"role": "user", "content": "How's the weather in Hangzhou and Shanghai?"}]
message = send_messages(messages)

In [4]:
message.tool_calls

[ChatCompletionMessageToolCall(id='call_75sAs0aOZq0wbDSZVZ05A6eB', function=Function(arguments='{"location": "Hangzhou, China"}', name='get_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_i2W7jymWUZucM6hrNfIMYIvC', function=Function(arguments='{"location": "Shanghai, China"}', name='get_weather'), type='function')]

### 多函数的调用

In [31]:
def get_weather(location):
    if location.lower() == "hangzhou":
        return "24℃, rain"
    elif location.lower() == "shanghai":
        return "25℃, cloudy"
    else:
        return "30℃, sunny"


def get_attractions(location):
    """
    返回指定城市的景点列表（模拟数据）
    """
    attractions_data = {
        "hangzhou": ["西湖", "灵隐寺", "雷峰塔", "宋城"],
        "shanghai": ["外滩", "东方明珠", "迪士尼乐园", "南京路步行街"],
    }

    # 统一小写，避免大小写问题
    city = location.lower()

    # 返回对应城市的景点，如果不存在则返回默认景点
    return attractions_data.get(city, attractions_data["default"])


available_functions = {"get_weather": get_weather,
                       "get_attractions": get_attractions}


def send_messages(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        # parallel_tool_calls = True
    )
    return response.choices[0].message


client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_attractions",
            "description": "Get popular attractions of a specified city",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. Hangzhou, Shanghai"
                    }
                },
                "required": ["location"]
            }
        }
    }
]
messages = [
    {"role": "user", "content": "How's the weather in Hangzhou and Shanghai? please recommend attractions in these cities.​"}]
message = send_messages(messages)

In [32]:
message.tool_calls

[ChatCompletionMessageToolCall(id='call_9JzvzHTo9wacKnuJKCSBRQfM', function=Function(arguments='{"location": "Hangzhou, China"}', name='get_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_eGPDT0bmWLuoM3wrjLGCv2dr', function=Function(arguments='{"location": "Shanghai, China"}', name='get_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_fxe2H30ruYUGt7mOCAR2AlyQ', function=Function(arguments='{"location": "Hangzhou"}', name='get_attractions'), type='function'),
 ChatCompletionMessageToolCall(id='call_qH5HzB5OjUp5DZgWjvT9dufh', function=Function(arguments='{"location": "Shanghai"}', name='get_attractions'), type='function')]

> 之前用的提示词：How's the weather in Hangzhou and Shanghai? please recommend attractions in cities without rain.
无论parallel_tool_calls设置True还是False，只调用了get_weath2次，我意识到错了，后面的调用取决于前面调用的结果，有依赖关系。
修改之后，2个tool的调用没有依赖关系。

In [21]:
def send_messages(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        parallel_tool_calls = False
    )
    return response.choices[0].message
messages = [
    {"role": "user", "content": "How's the weather in Hangzhou and Shanghai? please recommend attractions in these cities.​"}]
message = send_messages(messages)
message.tool_calls

[ChatCompletionMessageToolCall(id='call_Bk5yvkCE9KKdl1skNBpf00SS', function=Function(arguments='{"location": "Hangzhou, China"}', name='get_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_EfXejJhyV2XAvXjysxghKNaC', function=Function(arguments='{"location": "Shanghai, China"}', name='get_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_Fekn757do3rkiJw2TQBtiogc', function=Function(arguments='{"location": "Hangzhou"}', name='get_attractions'), type='function'),
 ChatCompletionMessageToolCall(id='call_JgZ8laOzKuPa082FEzGDTOGL', function=Function(arguments='{"location": "Shanghai"}', name='get_attractions'), type='function')]